In [4]:
import sympy

# -----------------------------------------------------------------------------
# 1. 初始化：定义所有变量
# -----------------------------------------------------------------------------
# 8个原始二元变量
p1, p2, q1, q2, c1, c2, c3, c4 = sympy.symbols('p1, p2, q1, q2, c1, c2, c3, c4')
original_vars = {p1, p2, q1, q2, c1, c2, c3, c4}

# 惩罚项系数M
M = sympy.Symbol('M')

# -----------------------------------------------------------------------------
# 2. 设计最优替换策略（新增4个辅助变量）
# -----------------------------------------------------------------------------
print("--- 优化策略：为4个关键二次对创建辅助变量 ---")

# 定义4个新的辅助变量
z1, z2, z3, z4 = sympy.symbols('z1, z2, z3, z4')
aux_vars = {z1, z2, z3, z4}

# 定义替换规则，这正是算法的核心
# z1 = p1*q1, z2 = p2*q2, z3 = p2*q1, z4 = p1*q2
substitutions = {
    p1*q1: z1,
    p2*q2: z2,
    p2*q1: z3,
    p1*q2: z4
}

print("已确定替换规则:")
for pair, z_var in substitutions.items():
    print(f"  {z_var} = {pair}")
print("\n总变量数将为 8 (原始) + 4 (新增) = 12。")


# -----------------------------------------------------------------------------
# 3. 执行替换并构建QUBO
# -----------------------------------------------------------------------------
# 定义原始函数 f
f_term1 = (2*p2 + 2*p1*q1 + 2*q2 - 8*c2 - 4*c1 + p1 + q1 - 3)**2
f_term2 = (2*q1 + 2*p2*q2 + 2*p1 + 2*c2 - 8*c4 - 4*c3 + p2*q1 + p1*q2 + c1 + 1)**2
f_term3 = (q2 + p2 + c3 + 2*c4 - 2)**2
f_original = f_term1 + f_term2 + f_term3

# 步骤A: 在展开前，先进行替换
print("\n--- 步骤A: 在展开前进行替换 ---")
f_substituted = f_original.subs(substitutions)
print("函数内部的二次项已被辅助变量替换。")

# 步骤B: 展开替换后的函数并应用二元特性
print("\n--- 步骤B: 展开并线性化幂次 ---")
# 因为内部已是线性，展开后直接是二次形式
H_main = f_substituted.expand()

# 应用 x^2 = x 规则 (对所有变量，包括辅助变量)
all_vars = original_vars.union(aux_vars)
power_subs_dict = {v**2: v for v in all_vars}
H_main = H_main.subs(power_subs_dict)
print("函数主体已转化为QUBO形式。")

# 步骤C: 构建惩罚项
print("\n--- 步骤C: 添加4个惩罚项 ---")
penalty_poly = 0
for pair, z_var in substitutions.items():
    v1, v2 = pair.args
    penalty_poly += M * (v1 * v2 - 2 * v1 * z_var - 2 * v2 * z_var + 3 * z_var)

# 最终的QUBO函数 H
H_qubo_optimized = (H_main + penalty_poly).expand()

# -----------------------------------------------------------------------------
# 4. 显示最终结果
# -----------------------------------------------------------------------------
print("\n--- 最终结果 ---")
print("已成功生成优化的QUBO函数 H！")

final_vars = H_qubo_optimized.atoms(sympy.Symbol)
print(f"\n最终函数 H 的总变量数为 {len(final_vars) - 1} (不计M):") # -1 to exclude M
print(sorted([str(v) for v in final_vars if v != M]))

print(f"\n函数 H 包含 {len(H_qubo_optimized.args)} 项。")
print("这个形式可以直接用于QUBO求解器。")

# 如果你想查看完整的（仍然非常巨大的）表达式，取消下面的注释
print("\n优化后的QUBO表达式 H:")
sympy.init_printing(use_unicode=True)
print(H_qubo_optimized)

--- 优化策略：为4个关键二次对创建辅助变量 ---
已确定替换规则:
  z1 = p1*q1
  z2 = p2*q2
  z3 = p2*q1
  z4 = p1*q2

总变量数将为 8 (原始) + 4 (新增) = 12。

--- 步骤A: 在展开前进行替换 ---
函数内部的二次项已被辅助变量替换。

--- 步骤B: 展开并线性化幂次 ---
函数主体已转化为QUBO形式。

--- 步骤C: 添加4个惩罚项 ---

--- 最终结果 ---
已成功生成优化的QUBO函数 H！

最终函数 H 的总变量数为 12 (不计M):
['c1', 'c2', 'c3', 'c4', 'p1', 'p2', 'q1', 'q2', 'z1', 'z2', 'z3', 'z4']

函数 H 包含 84 项。
这个形式可以直接用于QUBO求解器。

优化后的QUBO表达式 H:
M*p1*q1 + M*p1*q2 - 2*M*p1*z1 - 2*M*p1*z4 + M*p2*q1 + M*p2*q2 - 2*M*p2*z2 - 2*M*p2*z3 - 2*M*q1*z1 - 2*M*q1*z3 - 2*M*q2*z2 - 2*M*q2*z4 + 3*M*z1 + 3*M*z2 + 3*M*z3 + 3*M*z4 + 68*c1*c2 - 8*c1*c3 - 16*c1*c4 - 4*c1*p1 - 16*c1*p2 - 4*c1*q1 - 16*c1*q2 - 16*c1*z1 + 4*c1*z2 + 2*c1*z3 + 2*c1*z4 + 43*c1 - 16*c2*c3 - 32*c2*c4 - 8*c2*p1 - 32*c2*p2 - 8*c2*q1 - 32*c2*q2 - 32*c2*z1 + 8*c2*z2 + 4*c2*z3 + 4*c2*z4 + 120*c2 + 68*c3*c4 - 16*c3*p1 + 2*c3*p2 - 16*c3*q1 + 2*c3*q2 - 16*c3*z2 - 8*c3*z3 - 8*c3*z4 + 5*c3 - 32*c4*p1 + 4*c4*p2 - 32*c4*q1 + 4*c4*q2 - 32*c4*z2 - 16*c4*z3 - 16*c4*z4 + 44*c4 + 4*p1*p2 + 10*p1*